In [43]:
import pandas as pd
import sys
sys.path
sys.path.append('../models')
import numpy as np
import pickle
import random
import os
import math
import sim_functions_cleaner  as sf
import operator
import study
import time as time_module
import TS_personal_params_pooled as pp
import TS_global_params_pooled as gtp
from numpy.random import uniform

#sys.path.append('../simulation')
import TS_fancy_pooled 
import TS
#import TS_fancy_pooled 
import eta
import pooling_bandits as pb
import warnings 
warnings.simplefilter('ignore')
from sklearn import preprocessing
import tensorflow as tf
import gc

In [3]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
import logging
logging.getLogger("FFC").setLevel(logging.ERROR)

In [4]:
from pympler import muppy
all_objects = muppy.get_objects()
from pympler import summary


In [4]:
##for each person generate dates
##easiest to take from original data, this might best mimic actual situation 

In [5]:
#root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/processed/'

In [5]:
write_directory = '../temp'

In [17]:
def initialize_policy_params_TS(experiment,context_dimension):
    
    #global_p =gtp.TS_global_params(10,context_dimension) 
    
    
    global_p =gtp.TS_global_params(10,baseline_features=63,psi_features=[0,64], resp_features= 63)
    personal_p = pp.TS_personal_params()
    #global_p =gtp.TS_global_params(10,context_dimension)
    
    
    
    #global_p.mu_dimension = 64

    global_p.kdim = 194
    global_p.baseline_indices = [i for i in range(192)]
    #[0,1,2,3,4,5,6]
    global_p.psi_indices = [0,64]
    global_p.user_id_index = 192
    global_p.user_day_index = 193
    
    #global_p.baseline_features = [i for i in range(192)]
    global_p.psi_features = [0,64]
    
    
    
    
    global_p.update_period = 1
    
    
    #print(type(personal_p))
    
    for person in experiment.population.keys():
        experiment.population[person].root = '../../regal/murphy_lab/pooling/distributions/'
        initial_context = [0 for i in range(global_p.theta_dim)]
        personal_p.mus0[person]= global_p.get_mu0(initial_context)
        personal_p.mus1[person]= global_p.get_mu1(global_p.num_baseline_features)
        personal_p.mus2[person]= global_p.get_mu2(global_p.num_responsivity_features)
        
        personal_p.sigmas0[person]= global_p.get_asigma(len( personal_p.mus0[person]))
        personal_p.sigmas1[person]= global_p.get_asigma(global_p.num_baseline_features+1)
        personal_p.sigmas2[person]= global_p.get_asigma( global_p.num_responsivity_features+1)
        
        
        
        personal_p.batch[person]=[[] for i in range(len(experiment.person_to_time[person]))]
        personal_p.batch_index[person]=0
        
        #personal_p.etas[person]=eta.eta()
        
        personal_p.last_update[person]=experiment.person_to_time[person][0]
        
        
    return global_p ,personal_p     

In [8]:
#days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=45),freq='30T')

In [9]:
#ft = experiment.study_days[96]

In [10]:
#yesterday = ft-pd.DateOffset(days=1)
#end = yesterday.replace(hour=0,minute=0)
#start = experiment.study_days[0]
#end

In [239]:
def new_kind_of_simulation(experiment,policy=None,personal_policy_params=None,global_policy_params=None):
    

    #write_directory = '../../murphy_lab/lab/pooling/temp'
    experiment.last_update_day=experiment.study_days[0]
    
    get_m_times = []
    
    for time in experiment.study_days:
        if global_policy_params.decision_times>1000:
                print('avg m time')
                print(np.array(get_m_times).mean())
                return history
                break
        #if time> experiment.study_days[0]:
        #history  = pb.make_history(experiment)
        if time==experiment.last_update_day+pd.DateOffset(days=global_policy_params.update_period):
            experiment.last_update_day=time
            print('Global update', time,global_policy_params.decision_times, file=open('updates_{}_{}.txt'.format(len(experiment.population),global_policy_params.update_period), 'a'))
            if global_policy_params.decision_times>500:
                glob.last_global_update_time=time
                history =pb.make_history_one_hot(uniform(),glob,experiment)
                    #print(history[1])
                temp_params = pb.run(history[0],history[1],global_policy_params,gp_train_type = 'Static')
          
                global_policy_params.update_params(temp_params)
                #print(temp_params)
                global_policy_params.history = history
                
            ##update global context
            ##global context shared across all participants
        tod = sf.get_time_of_day(time)
        dow = sf.get_day_of_week(time)
        if time==experiment.study_days[0]:
            print('init weather')
            weather = sf.get_weather_prior(tod,time.month)
        elif time.hour in experiment.weather_update_hours and time.minute==0:
            weather = sf.get_next_weather(str(tod),str(time.month),weather)
            ##location depends on person 
            
        for person in experiment.dates_to_people[time]:
                dt=False
                action = 0 
                prob=0
                #1
                ##for every active person update person specific aspects of their context
                participant = experiment.population[person]
                #update global context variables
                participant.set_tod(tod)
                participant.set_dow(dow)
                participant.set_wea(weather)
                
                
                availability = (uniform() < 0.8)
                participant.set_available(availability)
                
                if time == participant.times[0]:
                    #get first location 
                    location = sf.get_location_prior(str(participant.gid),str(dow),str(tod))
                    participant.set_inaction_duration(0)
                    participant.set_action_duration(0)
                    participant.set_duration(0)
                    participant.set_dosage(0)
                    #personal_policy_params.etas[participant.pid]
                    
                    
                if time <= participant.times[1]:
                    steps_last_time_period = 0  
                    
                    ##set first pre-treatment, yesterday step count, variation and dosage
                else:
                    
                    if time.hour==0 and time.minute==0:
                        participant.current_day_counter=participant.current_day_counter+1
                    
                    #print(time)
                    steps_last_time_period = participant.steps
                
                 
                    #get var id
                    
                #if time.date() <= participant.times[0].date():
                    #steps_yesterday = 0    
                #else:
                    #steps_yesterday =  participant.find_yesterday_steps(time)
                    #steps_yesterday = sf.to_yid(steps_yesterday)
                steps_yesterday=0    
                if time.hour in experiment.location_update_hours:
                    location = sf.get_next_location(participant.gid,dow,tod,participant.get_loc())
                
                if time.date()>(participant.times[0]+pd.DateOffset(days=1)).date():
                  
                    if time.hour==0 and time.minute==0:
                        variation = 0
                        #participant.find_variation(time)
                else:
                    variation = 1 
                
                participant.set_loc(location)
      
                prob = -1
                if time in participant.decision_times:
                                     
                    
                    
                    
                    dt=True
                    action=0
                    
                    
                    if global_policy_params.decision_times>500 and global_policy_params.history!=None:
                     
                            ##change dimension of mu
                            startt = time_module.time()
                            temp = calculate_posterior(global_policy_params,\
                                                  participant.pid,participant.current_day_counter,\
                                                  history[0], history[1] )
                            
                            endt = time_module.time()
                            print(endt-startt)
                            get_m_times.append(endt-startt)
                    #print(temp[0].shape)
                    else:
                        #print('here')
                        temp = [personal_policy_params.mus2[participant.pid],personal_policy_params.sigmas2[participant.pid]]
                    mu_beta = temp[0]
                    Sigma_beta = temp[1]
                    personal_policy_params.update_mus(participant.pid,mu_beta,2)
                    personal_policy_params.update_sigmas(participant.pid,Sigma_beta,2)    
                    
                    
                    if policy==None:
                        action = sf.get_action(policy)
                        
                        
                        
                    elif policy=='TS':
                        #some context slice
                        
                        ##both f_one and g_one
                        one_hot_vector = pb.get_one_hot_encodings(global_policy_params,{'steps':steps,'weather':weather,'location':location,'ltps':participant.steps,'study_day':participant.current_day_counter,'decision_time':dt,'time':time,'avail':availability})
                        z = np.zeros(global_policy_params.num_responsivity_features+1)
                        z[0]=1
                        z[1:]=one_hot_vector
                        prob = TS.prob_cal_ts(z,0,personal_policy_params.mus2[participant.pid],personal_policy_params.sigmas2[participant.pid],global_policy_params)
                        action = int(uniform() < prob)
                            
                    
                    
                    if availability:
                    

                   
                    
                    
                        participant.update_dosage(action)
                    
                        context = [action,participant.gid,tod,dow,weather,sf.get_pretreatment(participant.steps),location,\
                              steps_yesterday,variation,sf.dosage_to_dosage_key(participant.dosage)]
                    
                        participant.steps_last_time_period = participant.steps
                        steps = sf.get_steps_action(context)
                        participant.steps = steps
                    else:
                        participant.steps_last_time_period = participant.steps
                        steps = sf.get_steps_no_action(participant.gid,tod,dow,location,weather,participant.steps)
                        participant.steps = steps

                
                #my_directory = '{}/participant_{}'.format(global_policy_params.write_directory,participant.pid)
                #if not os.path.exists(my_directory):
                #    os.makedirs(my_directory)
                # with open('{}/day_{}'.format(my_directory,global_policy_params.decision_times),'wb') as f:
                #    pickle.dump(context_dict,f)
                        
                        
                    global_policy_params.decision_times =   global_policy_params.decision_times+1
                
                    
                    
                else:
                        participant.steps_last_time_period = participant.steps
                        steps = sf.get_steps_no_action(participant.gid,tod,dow,location,weather,participant.steps)
                        participant.steps = steps     
                
                ##history:
                context_dict =  {'steps':steps,'action':action,'weather':weather,'location':location,\
                    'ltps':steps_last_time_period,'duration':participant.duration,\
                        'study_day':participant.current_day_counter,'decision_time':dt,'time':time,'avail':availability,'prob':prob}
                participant.history[time]=context_dict

In [232]:
def create_H(num_baseline_features,num_responsivity_features):
    ##for now have fixed random effects size one
    
    random_effect_one = [1]
    random_effect_two = [1]
    
    column_one = [1]
    column_one = column_one+[0]*num_baseline_features
    column_one = column_one+[0]
    column_one = column_one+[0]*num_responsivity_features
    column_one = column_one+[0]
    column_one = column_one+[0]*num_responsivity_features
    
    
    column_two = [0]
    column_two = column_two+[0]*num_baseline_features
    column_two = column_two+[1]
    column_two = column_two+[0]*num_responsivity_features
    column_two = column_two+[1]
    column_two = column_two+[0]*num_responsivity_features
    
    return np.transpose(np.array([column_one,column_two]))

In [233]:
def get_RT(y,X,sigma_theta,x_dim):
    
    to_return = [y[i]-np.dot(X[i][0:x_dim],np.ones(x_dim)) for i in range(len(X))]
    return np.array([i[0] for i in to_return])

In [234]:
def get_M(global_params,user_id,user_study_day,history):
  
  
    day_id =user_study_day
    #print(history)
    M = [[] for i in range(history.shape[0])]

    H = create_H(global_params.num_baseline_features,global_params.num_responsivity_features)
    for x_old_i in range(history.shape[0]):
        x_old = history[x_old_i]
        old_user_id = x_old[global_params.user_id_index]
        old_day_id = x_old[global_params.user_day_index]
        
        ##these indices all need to be parameters
        phi = np.array([x_old[i] for i in global_params.baseline_indices])
        
        t_one = np.dot(np.transpose(phi),global_params.sigma_theta)
        #first_terms.append(t_one)
        
        temp = np.dot(H,global_params.sigma_u)
        temp = np.dot(temp,H.T)
        temp = np.dot(np.transpose(phi),temp)
        temp = float(old_user_id==user_id)*temp
        t_two = temp
        #middle_terms.append(t_two)
        temp = np.dot(H,global_params.sigma_v.reshape(2,2))
        temp = np.dot(temp,H.T)
        temp = np.dot(np.transpose(phi),temp)
        temp = rbf_custom_np(user_study_day,old_day_id)*temp
        t_three = temp
        #print(user_study_day)
        
        #last_terms.append(t_three)
        term = np.add(t_one,t_two)
        
        term = np.add(term,t_three)
        #print(term.shape)
        #print(term)
        M[x_old_i]=term

    return np.array(M)

In [235]:
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(test))] for i in range(len(test))])

NameError: name 'rdayone' is not defined

In [217]:
users = np.array([1,2,3,4])

In [97]:
t = np.ma.masked_where(users==1, users).mask.astype(float)
np.diag(t)

array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [218]:
def get_M_faster(global_params,user_id,user_study_day,history):
  
  
    day_id =user_study_day
    #print(history)
    M = [[] for i in range(history.shape[0])]

    H = create_H(global_params.num_baseline_features,global_params.num_responsivity_features)
    
    phi = history[:,global_params.baseline_indices]
    ##should be fine
    t_one = np.dot(phi,global_params.sigma_theta)
    temp = np.dot(H,global_params.sigma_u)
    temp = np.dot(temp,H.T)
    temp = np.dot(phi,temp)
    

    
    
    user_ids = history[:,global_params.user_id_index]
    days_ids = history[:,global_params.user_day_index]
        
    my_days = np.ma.masked_where(user_ids==user_id, user_ids).mask.astype(float)
    user_matrix = np.diag(my_days)
    
    rho_diag = np.diag([rbf_custom_np(d,user_study_day) for d in days_ids])

    t_two = np.matmul(user_matrix,temp)

    temp = np.dot(H,global_params.sigma_v.reshape(2,2))
    temp = np.dot(temp,H.T)
    temp = np.dot(phi,temp)
    #temp = rbf_custom_np(user_study_day,old_day_id)*temp
    t_three = np.matmul(rho_diag,temp)
    term = np.add(t_one,t_two)
        
    term = np.add(term,t_three)
    
    
    
    return term

    #rdayone = [x[global_params.user_day_index] for x in X]
    #rdaytwo = rdayone
    #rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(len(X))] for i in range(len(X))])
    



In [219]:
m =get_M_Faster(glob,1,1,hist[0])

In [190]:
m.shape

(437, 192)

In [243]:
def calculate_posterior(global_params,user_id,user_study_day,X,y):
    H = create_H(global_params.num_baseline_features,global_params.num_responsivity_features)
    start_m = time_module.time()
    M = get_M(global_params,user_id,user_study_day,X)
    #M = get_M_faster(global_params,user_id,user_study_day,X)
    #print('M check')
    #print(M==M_two)
    #eqs = set([])
    #for i in range(M.shape[0]):
        #for j in range(M.shape[1]):
            #eqs.add(M[i][j]==M_two[i][j])
    #print(eqs)
    #print(M[0])
    #print(M_two[0])
    end_m = time_module.time()
    
    #print('get M')
    #print(end_m-start_m)
    mtime = end_m-start_m
    ##change this to be mu_theta
    ##is it updated?  the current mu_theta?
    start_m = time_module.time()
    adjusted_rewards =get_RT(y,X,global_params.mu_theta,global_params.theta_dim)
    end_m = time_module.time()
    #print('get R')
    #print(end_m-start_m)
    #print('current global cov')
    #print(global_params.cov)
    #.reshape(X.shape[0],X.shape[0])
    start = time_module.time()
    mu = get_middle_term(X.shape[0],global_params.cov,global_params.noise_term,M,adjusted_rewards,global_params.mu_theta)
    end = time_module.time()
    #print('middle')
    #print(end-start)
    start = time_module.time()
    
    sigma = get_post_sigma(H,global_params.cov,global_params.sigma_u.reshape(2,2),global_params.sigma_v.reshape(2,2),global_params.noise_term,M,X.shape[0],global_params.sigma_theta)

    end = time_module.time()
   # print('sigma')
    #print(end-start)
    
    
    return mu[-(global_params.num_responsivity_features+1):],[j[-(global_params.num_responsivity_features+1):] for j in sigma[-(global_params.num_responsivity_features+1):]],mtime

In [221]:
def get_middle_term(X_dim,cov,noise_term,M,adjusted_rewards,mu_theta):
    #M = get_M(global_params,user_id,user_study_day,history[0])
    
    ##change this to be mu_theta
    ##is it updated?  the current mu_theta?
    #adjusted_rewards =[history[1][i]-np.dot(history[0][i][0:6],np.ones(6)) for i in range(len(history[0]))]
    
    ##noise = noise_term * np.eye(X_dim)
    #print(noise.shape)
    #print(cov.shape)
    ##middle_term = np.add(cov,noise)
    
    ##middle_term = np.dot(M.T,np.linalg.inv(middle_term))

    ##middle_term = np.dot(middle_term,adjusted_rewards)
    ##return np.add(mu_theta,middle_term)
    noise = noise_term * np.eye(X_dim)
    #print(noise.shape)
    #print(cov.shape)
    #print('in get middle')
    middle_term = np.add(cov,noise)
    #print(middle_term)
    #print(M.shape)
    middle_term = np.matmul(M.T,np.linalg.inv(middle_term))
    #print(middle_term)
    middle_term = np.matmul(middle_term,adjusted_rewards)
    #print(middle_term)
    return np.add(mu_theta,middle_term)

In [222]:
def get_post_sigma(H,cov,sigma_u,sigma_v,noise_term,M,x_dim,sigma_theta):
    #M = get_M(global_params,user_id,user_study_day,history[0])
    
    ##change this to be mu_theta
    ##is it updated?  the current mu_theta?
    #adjusted_rewards =[history[1][i]-np.dot(history[0][i][0:6],np.ones(6)) for i in range(len(history[0]))]
    
    
    
    first_term = np.add(sigma_u,sigma_v)
    #print(first_term.shape)
    #print(H.shape)
    first_term = np.dot(H,first_term)
    #print(first_term.shape)
    first_term = np.dot(first_term,H.T)
    #print(first_term)
    
    noise = noise_term * np.eye(x_dim)
    #print(noise.shape)
    middle_term = np.add(cov,noise)
    #print(middle_term.shape)
    middle_term = np.dot(M.T,np.linalg.inv(middle_term))
    #print(middle_term.shape)
    middle_term = np.dot(middle_term,M)
    #print(middle_term.shape)
    last = np.add(sigma_theta,first_term)
    last = np.subtract(last,middle_term)
    
    return middle_term

In [223]:
def rbf_custom_np( X, X2=None):
    #print(X)
    #print(X2)
    if X2 is None:
        X2=X
    return math.exp(-((X-X2)**2)/100.0)

In [244]:
experiment = study.study('../../Downloads/distributions/',50)

In [245]:
glob,personal = initialize_policy_params_TS(experiment,11)

In [ ]:
start = time.time()
hist = new_kind_of_simulation(experiment,'TS',personal,glob)
end = time.time()
print(end-start)

init weather


called
0.07181692123413086
0.08944296836853027
0.08600902557373047
0.11975383758544922
0.08492779731750488
0.08477997779846191
0.11690807342529297
0.08603096008300781
0.08283805847167969
0.10892200469970703
0.08609223365783691
0.09116005897521973
0.1133110523223877
0.09111809730529785
0.09688925743103027
0.10120511054992676
0.09074020385742188
0.08600497245788574
0.10743498802185059
0.08641815185546875
0.0876626968383789
0.13467979431152344
0.08849787712097168
0.09023809432983398
0.11114788055419922
0.08579778671264648
0.10074591636657715
0.10945606231689453
0.08667635917663574
0.08313488960266113
0.12317276000976562
0.0866849422454834
0.08752632141113281
0.1125643253326416
0.1126861572265625
0.12467026710510254
0.08702516555786133
0.08598184585571289
0.11239790916442871
0.08790087699890137
0.08492398262023926
0.10686993598937988
0.08791399002075195
0.0868978500366211
0.1267070770263672
0.08337521553039551
0.11045074462890625
0.11017203330993652
0.08379268646240234
0.08832001686096191


called
0.09555411338806152
0.18947291374206543
0.2555811405181885
0.19919204711914062
0.17460298538208008
0.16916608810424805
0.14250922203063965
0.17573928833007812
0.1663353443145752
0.1396806240081787
0.1703779697418213
0.14178991317749023
0.1581249237060547
0.25753116607666016
0.22494983673095703
0.15829682350158691
0.19249510765075684
0.14999604225158691
0.21889996528625488
0.25925111770629883
0.16013717651367188
0.18766999244689941
0.14217114448547363
0.16807985305786133
0.2263031005859375
0.14232182502746582
0.22385787963867188
0.1618649959564209
0.16923284530639648
0.13379693031311035
0.16636300086975098
0.1512923240661621
0.19610023498535156
0.14637494087219238
0.14870285987854004
0.15116214752197266
0.14579010009765625
0.14686799049377441
0.16951513290405273
0.14769721031188965
0.14424872398376465
0.1300971508026123
0.1444079875946045
0.14145302772521973
0.1324012279510498
0.12792277336120605
0.20281100273132324
0.13166499137878418
0.14735007286071777
0.1297159194946289
0.138

called
0.10471677780151367
0.13507294654846191
0.14147019386291504
0.12668967247009277
0.14145612716674805
0.12060403823852539
0.13032102584838867
0.12108087539672852
0.13663220405578613
0.1174159049987793
0.12712478637695312
0.13386774063110352
0.15077996253967285
0.12547993659973145
0.13669204711914062
0.12347221374511719
0.13215994834899902
0.11542987823486328
0.1278669834136963
0.11330890655517578
0.12819194793701172
0.1360459327697754
0.1357109546661377
0.14371418952941895
0.14808011054992676
0.11265897750854492
0.12844514846801758
0.12441802024841309
0.12807393074035645
0.11846494674682617
0.12110280990600586
0.11985111236572266
0.15872502326965332
0.1268777847290039
0.1749439239501953
0.17600703239440918
0.15682291984558105
0.13361001014709473
0.13524508476257324
0.11868882179260254
0.15411901473999023
0.1388568878173828
0.12493300437927246
0.12856221199035645
0.13280487060546875
0.12749981880187988
0.1611919403076172
0.11646676063537598
0.13578510284423828
0.12535810470581055
0

called
0.11365795135498047
0.12376809120178223
0.14690589904785156
0.22717595100402832
0.2424907684326172
0.20804095268249512
0.18194818496704102
0.17516326904296875
0.13550305366516113
0.2438488006591797
0.24824190139770508
0.2554948329925537
0.31704187393188477
0.2522737979888916
0.3300619125366211
0.3385608196258545
0.2531471252441406
0.19794511795043945
0.21591711044311523
0.18107199668884277
0.24939680099487305
0.2524607181549072
0.17828989028930664
0.20043182373046875
0.1746811866760254
0.23002195358276367
0.24660277366638184
0.18267488479614258
0.22025609016418457
0.17154312133789062
0.2369978427886963
0.16979384422302246
0.20418310165405273
0.1728219985961914
0.24175000190734863
0.20390605926513672
0.23684215545654297
0.2432410717010498
0.19273090362548828
0.18388104438781738
0.22551488876342773
0.2685718536376953
0.1847529411315918
0.18551087379455566
0.18197917938232422
0.19720101356506348
0.16553592681884766
0.2229142189025879
0.17825007438659668
0.1985759735107422
0.1683857

called
0.12191104888916016
0.15949487686157227
0.16988396644592285
0.13845205307006836
0.18718481063842773
0.1773090362548828
0.15010714530944824
0.13174223899841309
0.17620301246643066
0.14252376556396484
0.14521503448486328
0.1346759796142578
0.19161701202392578
0.14858317375183105
0.15459036827087402
0.16986823081970215
0.15880894660949707
0.14610624313354492
0.19699621200561523
0.1450488567352295
0.15436911582946777
0.14548683166503906
0.18094897270202637
0.14809012413024902
0.17880487442016602
0.16816997528076172
0.15250587463378906
0.14096808433532715
0.2010810375213623
0.16436004638671875
0.16582679748535156
0.18813085556030273
0.17233014106750488
0.1760389804840088
0.19428086280822754
0.1506822109222412
0.16597485542297363
0.14214301109313965
0.19274020195007324
0.1449449062347412
0.16972899436950684
0.18204522132873535
0.19074296951293945
0.144927978515625
0.1476917266845703
0.15324711799621582
0.16757607460021973
0.18430209159851074
0.15305376052856445
0.14172077178955078
0.1

##### all_steps  = [
]
for p in experiment.population.values():
    all_steps.extend([v['steps'] for v in p.history.values()])

In [72]:
#o
20.15853214263916

#faster

#with more data
159
0.0390877393575815

0

In [ ]:
[223.62212776908694,]

In [17]:
all_steps  = set([])
for p in experiment.population.values():
    all_steps.update([v['action'] for v in p.history.values()])
all_steps

{0, 1}

In [53]:
g = get_history(write_directory,1)

In [11]:
def create_phi_new(history_dict,pi,global_params):
    #these things will be accessed by the global params
    indices = ['weather','location']
    g0 = ['location']
    f1=['ltps']
    
    ##returns phi and psi indices
    ##this could be a bit faster not appending all the time
    all_data = []
    steps=[]
    for user_id,history in history_dict.items():
        #history = d.history
        #history_keys = sorted(history)
        for hk,h in history.items():
            
            h = history[hk]
            if h['decision_time']:
                v = [1]
                v.extend([h[i] for i in indices])
                v.append(pi*1)
                v.extend([pi*h[i] for i in f1])
                action = h['action']
                if action<0:
                    action=0
                
                v.append((action-pi)*1)
                v.extend([(action-pi)*h[i] for i in f1])
                v.append(action)
                v.append(float(user_id))
                v.append(float(h['study_day']))
                all_data.append(v)
                steps.append(h['steps'])
    return all_data,steps

In [55]:
p = create_phi_new(g,.6,glob)

In [12]:
def make_history_new(write_directory,pi,glob):
    g = get_history(write_directory,glob.decision_times)
    ad = create_phi_new(g,pi,glob)
    if len(ad[0])==0:
        return [[],[]]
    
    new_x = preprocessing.scale(np.array(ad[0]))
    new_y = preprocessing.scale(np.array(ad[1]))
    y = np.array([[float(r)] for r in new_y])
    X = new_x
    return [X,y]

In [49]:
np.array(gr[0][0]).mean()

446.51003316358634

In [7]:
np.array(means).mean()

137.29867338336618

In [38]:
gr[1][0]

[2, 0, 0, 0, 2, 1, 0, 1, 1]

In [13]:
get_today_variance(gr[0],gr[1],gr[2])

False


0.0

In [14]:
def get_all_day_slices(first,last):
    slices = []
    day = first
    
    while day<last:
        slice_one = [day]
        #slice_one.append(day.replace(hour=23,minute=30))
        
        day_two = day+pd.DateOffset(days=1)
        slice_one.append(day_two)
        slices.append(slice_one)
        day=day_two
    return slices

In [8]:
def transform_to_hour(steps):
    to_return = []
    for i in range(0,len(steps)-1,2):
        to_return.append(steps[i]+steps[i+1])
    return to_return

In [35]:
def day_slices_to_median(day_slices,day_lookup,all_steps):
    days_steps = []
    for ds in day_slices:
        steps = all_steps[day_lookup[ds[0]]:day_lookup[ds[1]]]
       
        steps = transform_to_hour(steps)
        print(len(steps))
        days_steps.append(np.array(steps).std())

    return np.median(np.array(days_steps))
        

In [19]:
x=get_all_day_slices(gr[3],gr[4])

In [43]:
gr[4]

Timestamp('2015-07-29 00:00:00', freq='30T')

In [13]:
[1]+[2]+[3,4]

[1, 2, 3, 4]

In [3]:
    last_index = dd[one_week_ago.date()][0]
    
    day_slices=get_all_day_slices(last_index,dd[i.date()][0])
    
    median = day_slices_to_median(day_slices,gr[2],gr[5])

You can pass in a bunch of decision times. I just tested with one. 

Right now I think the number of people is being totally ignored. This needs to be fixed. 

In [12]:
x=sim_functions.simulate_run(num_people,temp_indices,decision_times)

NameError: name 'temp_indices' is not defined

In [14]:
np.array(x).mean()
#np.array(x).std()

207.01370855122792

In [15]:
np.array(x).mean()
np.array(x).std()

267.8619020654599

In [ ]:
2-2-0-1-1-0-0-1

In [5]:
root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/temporary/'
data_root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/'


In [9]:
with open('../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/steps_both_groups_logs_dosage_estf.pkl'.format(),'rb') as f:
    df = pickle.load(f)

In [10]:
df.keys()

dict_keys(['2-0-2-1-2-1-0-3-18', '2-1-1-1-0-1-0-3-77', '2-0-0-0-3-3-0-3-9', '2-0-2-0-2-2-0-4-37', '2-1-2-1-0-1-0-4-70', '2-0-1-0-2-0-0-4-35', '2-1-2-1-0-1-0-3-13', '2-0-2-2-2-0-0-4-48', '2-0-1-1-2-1-0-3-28', '2-0-0-2-1-1-0-3-8', '2-0-2-0-2-0-0-3', '2-1-1-1-0-2-0-4-78', '2-0-1-1-2-1-0-3-31', '2-0-1-1-1-1-0-4-70', '2-0-1-2-1-3-0-4-66', '2-0-2-1-0-1-0-3-31', '2-0-1-2-0--1-0-4-25', '1-0-2-0-3-0-0-4-16', '2-1-2-0-3-2-0-4-3', '2-0-0-1-0-1-0-3', '2-1-1-1-1-1-0-3-18', '2-1-1-0-1-1-0-4-53', '2-1-2-2-1-1-0-4-35', '2-0-1-2-0--1-0-3-31', '2-0-1-2-0-3-0-4-64', '2-0-2-1-1-2-0', '1-1-1-0-3-3-0-3-38', '2-0-0-1-2-1-0-4-77', '2-0-2-2-1-1-1', '2-0-1-2-1-2-0', '2-0-2-2-0-2-1-3-3', '2-0-1-1-1-1-1-3-9', '1-0-2-0-0-0-0-4-94', '2-0-2-1-1-2-0-3-22', '2-0-1-2-0-0-1-4-9', '2-0-2-0-2-2-0-4-39', '2-1-1-1-2-0-0-4-91', '2-0-2-1-0-2-0-3-21', '2-0-1-0-0-0-0-4-38', '2-0-2-1-0-3-0-3-96', '2-0-2-0-0-1-1-4-1', '2-0-1-1-2-0-0-3-47', '2-0-1-1-1-2-0-4-9', '2-0-1-0-1-1-0-4-30', '2-0-1-1-0-2-0-3-53', '2-1-1-1-0-0-0-4-95', '2-1